In [1]:
pip install demucs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [7]:
pip install dotenv


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
API=os.getenv("API")
ELEVEN=os.getenv("ELEVEN")

In [6]:
!pip install openai-whisper google-generativeai torchaudio elevenlabs simpleaudio

import whisper
import torchaudio
import google.generativeai as genai
import IPython.display as ipd
from elevenlabs.client import ElevenLabs
import tempfile
import wave
from google.colab import files
import subprocess
import shutil

uploaded = files.upload()
filename = next(iter(uploaded))
waveform, sr = torchaudio.load(filename)
# MATCHING FREQUENCY OF WAV TO BE COMPATIBLE WITH WHISPER
if sr != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
    waveform = resampler(waveform)
    sr = 16000

clean_path = "cleaned.wav"

torchaudio.save(clean_path, waveform, sr)
# -----------------------DEMUCS PIPELINE FOR BACKGROUND FILTERING------------------------------#
subprocess.run([
    "demucs",
    "--two-stems", "vocals",
    "-n", "mdx_extra",
    "-o", ".",
    "cleaned.wav"
])
shutil.copy("mdx_extra/cleaned/vocals.wav", "vocals.wav")
# ---------------------------------------------------------------------------------------------
clean_path= "vocals.wav"
# -----------------------STT PIPELINE------------------------------#

asr_model = whisper.load_model("large-v3", device="cuda")
result = asr_model.transcribe(clean_path, language='sa')
sanskrit_text = result["text"]
print("Sanskrit Transcription:\n", sanskrit_text)
# -----------------------------------------------------------------#

api_key = API
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.0-flash")
# -----------------------CONVERSATION PIPELINE------------------------------#

target_language = "Sanskrit"
prompt = (
    f"You are a conversational AI that understands Sanskrit. A user will speak to you in Sanskrit. "
    f"Reply naturally and appropriately in {target_language}, based on what the user is saying. "
    f"Do not translate or explain the Sanskrit — just respond as if you're talking to the user in a conversation. "
    f"User: {sanskrit_text}"
)
response = model.generate_content(prompt)
converted = response.text
print("\nTranslation:\n", converted)

eleven_api_key = ELEVEN
client = ElevenLabs(api_key=eleven_api_key)
# -----------------------TTS PIPELINE------------------------------#

audio_generator = client.text_to_speech.convert(
    text=converted,
    voice_id="1qEiC6qsybMkmnNdVMbK",
    model_id="eleven_multilingual_v2",
    output_format="mp3_44100_128"
)

with open("output.mp3", "wb") as f:
    for chunk in audio_generator:
        f.write(chunk)
ipd.display(ipd.Audio("output.mp3"))
# ------------------------------------------------------------------#


Saving voice.wav to voice.wav


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:69: UserWarning: /root/.cache/whisper/large-v3.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(
100%|█████████████████████████████████████| 2.88G/2.88G [00:42<00:00, 73.4MiB/s]


Sanskrit Transcription:
  नमस्ते भवता नामाकिम्

Translation:
 नमस्ते! मम नाम कृत्रिमबुद्धिः। भवता किं नाम?

